In [702]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import regex as re
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.sql import text
from sqlalchemy.ext.declarative import declarative_base
from pathlib import Path

Base = declarative_base()

insurance_df = pd.read_csv(r'C:\Users\ormis\Desktop\Project-4-Team-2\insurance.csv')

insurance_df.head()

C:\Users\ormis\AppData\Local\Temp\ipykernel_13412\3210975671.py:13: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [703]:
# class Insurance(Base):
#     __tablename__ = 'health_care'
#     extend_existing=True
#     Index = Column(Integer, primary_key=True, autoincrement=True)
#     age = Column(Integer)
#     sex = Column(String)
#     bmi = Column(Float)
#     children = Column(Integer)
#     smoker = Column(String)
#     region = Column(String)
#     charges = Column(Float)

In [704]:
# Base.metadata.tables

In [705]:
# engine = create_engine('sqlite:///SQL Table Creation.sqlite').connect()

In [706]:
# Base.metadata.create_all(engine)

In [707]:
# from sqlalchemy.orm import sessionmaker

# Session = sessionmaker(bind=engine)
# session = Session()

In [708]:
# import_data = []

# for index, row in insurance_df.iterrows():
#     row_dict = {
#         'Index': index
#         }
#     import_data.append(row_dict)

# import_data

In [709]:
# import_data=[]

# for index, row in insurance_df.iterrows():
#     row_dict= {
#         'Index' : index,
#         'Age': row['age'],
#         'Sex': row['sex'],
#         'BMI': row['bmi'],
#         'Children': row['children'],
#         'Smoker': row['smoker'],
#         'Region': row['region'],
#         'Charges': row['charges']
#     }
# import_data.append(row_dict)

# final_df = pd.DataFrame(import_data)

In [710]:
# for row in import_data:
#     print(row['Index'])
#     try:
#         exist = session.query(Insurance).filter(Insurance.Index == row['Index']).first()
#     except Exception as e:
#         print(f"Error: {row['Index']}, Error: {e}")
#         continue

#     if exist:
#         continue

#     else:
#         new_entry = Insurance(
#             age= row['Age'],
#             sex= row['Sex'],
#             bmi= row['BMI'],
#             children= row['Children'],
#             smoker= row['Smoker'],
#             region= row['Region'],
#             charges= row['Charges']
#         )
#         session.add(new_entry)

# session.commit()

In [711]:
# final_df

In [712]:
insurance_cat = insurance_df.dtypes[insurance_df.dtypes == 'object'].index.tolist()

In [713]:
insurance_df[insurance_cat].nunique()

sex       2
smoker    2
region    4
dtype: int64

In [714]:
enc = OneHotEncoder(sparse_output=False)

encode_df = pd.DataFrame(enc.fit_transform(insurance_df[insurance_cat]))

encode_df.columns = enc.get_feature_names_out(insurance_cat)

encode_df.head()

,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0


In [715]:
insurance_df = insurance_df.merge(encode_df, left_index=True, right_index=True)

insurance_df = insurance_df.drop(insurance_cat, axis=1)

insurance_df.head()

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,18,33.770,1,1725.55230,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2,28,33.000,3,4449.46200,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,33,22.705,0,21984.47061,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4,32,28.880,0,3866.85520,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0


In [772]:
y = insurance_df['region_southwest'].values

X = insurance_df.drop(['age', 'bmi', 'children', 'sex_female', 'sex_male', 'smoker_yes', 'region_northeast', 'region_northwest', 'region_southeast', 'region_southwest'], axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=12)

In [773]:
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)

X_test_scaled = X_scaler.transform(X_test)

In [774]:
region_features = len(X_train[0])

hidden_nodes_layer1 = 8

hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1 , input_dim=region_features, activation='relu'))

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

nn.summary()

c:\Users\ormis\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_149 (Dense)               │ (None, 8)              │            24 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_150 (Dense)               │ (None, 5)              │            45 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_151 (Dense)               │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 75 (300.00 B)

 Trainable params: 75 (300.00 B)

 Non-trainable params: 0 (0.00 B)

In [775]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [780]:
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7550 - loss: 0.5544
Epoch 2/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7756 - loss: 0.5327
Epoch 3/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.7261 - loss: 0.5872
Epoch 4/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7324 - loss: 0.5780
Epoch 5/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7348 - loss: 0.5790
Epoch 6/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7449 - loss: 0.5673
Epoch 7/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7332 - loss: 0.5778
Epoch 8/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7584 - loss: 0.5532
Epoch 9/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7611 - loss: 0.5482
Epoch 10/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7442 - loss: 0.5652
Epoch 11/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7634 - loss: 0.5471
Epoch 12/50
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7687 -

In [781]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

11/11 - 0s - 15ms/step - accuracy: 0.7791 - loss: 0.5269
Loss: 0.5269036889076233, Accuracy: 0.779104471206665


In [782]:
region_features = len(X_train[0])

hidden_nodes_layer1 = 18

hidden_nodes_layer2 = 11

nn2 = tf.keras.models.Sequential()

nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1 , input_dim=region_features, activation='relu'))

nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

nn2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

nn2.compile(loss='binary_crossentropy', optimizer='AdamW', metrics=['accuracy'])

fit_model2 = nn2.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.4437 - loss: 0.7236
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7257 - loss: 0.6270
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7490 - loss: 0.5855
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7534 - loss: 0.5631
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7453 - loss: 0.5718
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7503 - loss: 0.5636
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7278 - loss: 0.5866
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7624 - loss: 0.5511
Epoch 9/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7563 - loss: 0.5571
Epoch 10/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7466 - loss: 0.5693
Epoch 11/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7501 - loss: 0.5629
Epoch 12/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accurac

In [783]:
model_loss, model_accuracy = nn2.evaluate(X_test_scaled, y_test, verbose=2)

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

11/11 - 1s - 80ms/step - accuracy: 0.7791 - loss: 0.5278
Loss: 0.5278337597846985, Accuracy: 0.779104471206665
